The below section of code reads in 'FPL_Database.csv' (which is every players data from the previous season) and converts their points to account for position changes if applicable. This is then merged with 'players.csv' (which is a list of players in the current years game) and finally we end up with two csv which we manually check called 'true.csv' and 'false.csv'. True contains all the players in this years game that were also in last years. False contains players in this years game that were not in last years so we do a manual check on this file to make sure this is correct (and manually update if necessary) and then add all of these players to 'true.csv' so we have a complete player list matching 'players.csv'. We then download expected points csv from fplreview.com and merge this with 'true.csv' to create our final database file, 'player_dataset.csv'.

In [2]:
import pandas as pd
import numpy as np

In [73]:
df= pd.read_csv('FPL_Database.csv')

In [74]:
# replace position values with Text values
df['Position'] = df['Position'].replace([1, 2, 3, 4], ['GK', 'DEF', 'MID', 'FWD']) 

In [75]:
# add extra columns for appearance and goals conceded.
df['>60mins'] = np.where(df['minutes'] > 59, 1,0)
df['<60mins'] = np.where((df['minutes'] < 60) & (df['minutes'] > 0), 1,0)
df['2 goals conceded'] = np.where(df['goals_conceded'] > 1, 1,0)

In [76]:
# drop unneccessary columns
df.drop(['Round','Team ID','Short Name','goals_conceded','value'],axis=1,inplace=True)

In [77]:
# group all GW together to get each player total stats for the season
df_grouped = df.groupby(['Player ID','Player Name','Position','Team Name'],as_index=False).sum()

In [78]:
# read in new seasons player list
df1 = pd.read_csv('players.csv')

In [79]:
# merge df1 and df_grouped on Name and team to avoid mishaps with players with same name on diff teams. 
# will be additional false names as some players will move clubs but need to address manually as ID different each year.
df_merged = pd.merge(df1, df_grouped, on=['Player Name','Team Name'])

In [80]:
# drop column not needed
df_merged.drop(['Player ID'],axis=1,inplace=True)

In [81]:
# rename some columns
df_merged.rename(columns={'Position_x':'new_pos', 'Position_y':'old_pos'}, inplace=True)

In [82]:
# Check if player and team name is in the merged df. This will add a true/false column into df1
# All false players will need data added manually.
df1 = df1.assign(Result=(df1['Player Name'].isin(df_merged['Player Name']) & (df1['Team Name'].isin(df_merged['Team Name']))))

In [83]:
# create df of players whose data needs to be added manually.
df_false = df1[df1['Result'] == False]

In [84]:
# get dataset of all players with a position change
position_change = df_merged.loc[df_merged['new_pos'] != df_merged['old_pos']]

In [85]:
# re-evaluate points based on position change
# use .copy to prevent SettingswithCopyWarnings.
def_to_mid = position_change.loc[(position_change['old_pos'] == "DEF") & (position_change['new_pos'] == "MID")].copy()
mid_to_def = position_change.loc[(position_change['old_pos'] == "MID") & (position_change['new_pos'] == "DEF")].copy()
mid_to_fwd = position_change.loc[(position_change['old_pos'] == "MID") & (position_change['new_pos'] == "FWD")].copy()
fwd_to_mid = position_change.loc[(position_change['old_pos'] == "FWD") & (position_change['new_pos'] == "MID")].copy()

In [86]:
# evaluate the points
def_to_mid['total_points'] = def_to_mid['total_points'] - def_to_mid['goals_scored'] - (def_to_mid['clean_sheets'] * 3) + def_to_mid['2 goals conceded']
mid_to_def['total_points'] = mid_to_def['total_points'] + mid_to_def['goals_scored'] + (mid_to_def['clean_sheets'] * 3) - mid_to_def['2 goals conceded']
mid_to_fwd['total_points'] = mid_to_fwd['total_points'] - mid_to_fwd['goals_scored'] - mid_to_fwd['clean_sheets']
fwd_to_mid['total_points'] = fwd_to_mid['total_points'] + fwd_to_mid['goals_scored'] + fwd_to_mid['clean_sheets']

In [87]:
# merge four position change df's together
position_change_merge = pd.concat([def_to_mid, mid_to_def,mid_to_fwd,fwd_to_mid], axis=0)

In [88]:
# update original merged df with new calculated values
df_merged.update(position_change_merge)

In [89]:
# drop unnecessary columns. Only concerned with total points and value
df_merged.drop(['goals_scored','assists','clean_sheets','own_goals','yellow_cards','red_cards','bonus','penalties_saved','penalties_missed','saves','>60mins','<60mins','2 goals conceded'],axis=1,inplace=True)

In [90]:
# rename position column. Leave 'Old_pos' for manual check needed below and delete then.
df_merged.rename(columns={"new_pos":'Position'}, inplace=True)

In [91]:
# if minutes played > 1000 then points per minute is calculated points/mins
df_merged.loc[df_merged['minutes'] > 1000, 'points_per_minute'] = df_merged['total_points']/df_merged['minutes']

In [92]:
#create two csv true and false that need to be manually merged together to check for data error, names changes etc in FPL.
df_false.to_csv('false.csv',index=False)
df_merged.to_csv('true.csv',index=False)

Next step is to manually check 'false.csv' to see if any of those players were in the league last season with different teams or names and manually update 'true.csv'. Also do quality checks on data, no duplicate names or missing players etc. FPL sometimes have players on same team with exact same name etc. Make sure all players in 'players.csv' are contained in either false.csv or true.csv. Finally add all false.csv players with 0 total points to 'true.csv'. Once this done false.csv can be deleted.

In [4]:
# Read in true csv
df_true= pd.read_csv('true.csv')

In [5]:
# drop column not needed
df_true.drop(['old_pos'],axis=1,inplace=True)

In [6]:
# Read in fplreview csv
df_fplreview= pd.read_csv('fplreview.csv')

In [7]:
# merge df_true and df_fplreview
df_final = pd.merge(df_true, df_fplreview, left_on=['Player Name', 'Team Name'],
                   right_on= ['Name', 'Team'], 
                   how = 'left')

In [8]:
#drop unnecessary columns
df_final.drop(['Pos','Name','BV','SV','Team'],axis=1,inplace=True)

In [9]:
# replace any NaN values with 0
df_final.replace(np.NaN, 0, inplace=True)

In [11]:
#output final dataset to csv 
df_final.to_csv('player_dataset.csv',index=False)

Can now delete 'true.csv' and cut 'player_dataset.csv' into main directory